<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/gradient-descent/notebooks/Housing_proces_kaggle_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder,\
Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoLarsCV, LinearRegression
import itertools
from random import shuffle

In [51]:
pd.set_option('display.max_rows', 100)
pd.set_option('max_colwidth', 80)

In [52]:
raw_df_train = pd.read_csv('https://raw.githubusercontent.com/Dimildizio/DS_course/main/gradient-boosting/data/train.csv', index_col=0)
raw_df_test = pd.read_csv('https://raw.githubusercontent.com/Dimildizio/DS_course/main/gradient-boosting/data/test.csv', index_col=0)


In [53]:
def improve_cats(dataframe) -> pd.DataFrame:
  df = dataframe.copy()
  for col in df.columns:
    if df[col].dtype == 'int64':
      df[col] = df[col].astype('int32')
    elif df[col].dtype == 'float64':
      df[col] = df[col].astype('float32')
    elif df[col].dtype == 'object':
      df[col] = df[col].astype('category')
    else:
      print('Unknown data type')
      return
  return df

In [54]:
df = improve_cats(raw_df_train) #pd.concat([df_train_1, df_test_1])
df_test = improve_cats(raw_df_test)

In [149]:
dates_frames  = ['YearBuilt', 'YeatRemodAdd', 'GarageYrBlt', "YrSold",'MoSold']

In [150]:
def cut_iqr(df, col, mult):
  d=df[col].describe()
  val =(d['50%'] + (d['75%']-d['25%'])) * mult
  return df[df[col] <= val]

In [168]:
def plotme(df, col):
    if col != 'SalePrice':
      sns.scatterplot(y = df['SalePrice'], x = df[col])

In [152]:
#we need to remove empty values in categories if there are ones 
def cut_cats(df):
  for catcol in df.columns:
    if df[catcol].dtype == 'category':
      df[catcol].cat.remove_unused_categories() 
  return df

def create_mt_catcol(df_new):
  df = df_new.copy()
  for col in df.columns:
      if df[col].dtype == 'category':
        if df[col].isna().any():
          df[col] = df[col].cat.add_categories(['MISSING'])
          df[col] = df[col].fillna('MISSING')
  cut_cats(df)
  return df

In [154]:
#Change after we get encoding
def get_valid_cols(df, ok_cols, to_drop=[]):
  return [x for x in df.columns if df[x].dtype not in ['category', 'object'] and x not in ok_cols+to_drop]

In [155]:
def split_data(df, target):
  X = df.drop(target, axis=1)
  y = df[target]
  return train_test_split(X, y, test_size=0.2, random_state=42)
  

In [156]:
#Could run separate times for y_pred_train, y_pred_test but training two could take longer 
def run_model(X_train, y_train, X_test, model = LinearRegression):
  pipe = make_pipeline(StandardScaler(), model())
  pipe.fit(X_train, y_train)

  y_pred_train = pipe.predict(X_train)
  y_pred_test = pipe.predict(X_test)
  return y_pred_train, y_pred_test

In [157]:
def split_run_test(df, target = 'SalePrice'):
  X_train, X_test, y_train, y_test = split_data(df, target)
  y_pred_train, y_pred_test = run_model(X_train, y_train, X_test)
  print_scores(y_train, y_test, y_pred_train, y_pred_test, print_train=True)

In [158]:
def print_scores(y_train, y_test, y_pred_train, y_pred_test, print_train=False):
    #print('Column: ', col)
    if print_train:
      print('train: r2', r2_score(y_train, y_pred_train))
      print("train: root mean squared error:", np.sqrt(mean_squared_error(y_pred_train, y_train)))
      print('Test:')
    print('test: r2', r2_score(y_test, y_pred_test))
    print("test: root mean squared error:", np.sqrt(mean_squared_error(y_pred_test, y_test)))
    print()

In [159]:
def find_best_param(dataset: pd.DataFrame, valid_cols: list, ok_cols: list, to_drop = [], r2_threshold = 0.67, target = "SalePrice"):
  dataset = dataset.copy()
  best_col = ['', r2_threshold]
  
  for col in valid_cols:
    #old good col + a test col
    test_cols = ok_cols + [col]
    df = dataset[test_cols]
  
    try:
      X_train, X_test, y_train, y_test = split_data(df, target)
      y_pred_train, y_pred_test = run_model(X_train, y_train, X_test)

      r2 = r2_score(y_test, y_pred_test) #compare_scores(y_train, y_test, y_pred_train, y_pred_test, r2_threshold)
      if r2 > r2_threshold:
        r2_threshold = r2
        best_col = [col, r2]
        #print_scores(y_train, y_test, y_pred_train, y_pred_test)
    except ValueError:
      continue
      #print(col, 'has NaN')
  
  result = ok_cols + [best_col[0]]
  #print('Best result:', result, best_col[1])
  return best_col


In [450]:
def get_best_cols(df, ok_cols: list, to_drop =[],  r2_threshold=0.67):
  valid_cols = get_valid_cols(df_cats, ok_cols)#
  valid_nodups = valid_cols.copy()
  shuffle(valid_cols)
  #best_result = r2_threshold
  for col in valid_cols:
    #print(new_valid)
    best_col, r2 = find_best_param(df_cats, valid_nodups, ok_cols, r2_threshold)
    if r2 - r2_threshold > 0.001:
      r2_threshold = r2
      ok_cols.append(best_col)
      valid_nodups.remove(col)
  return ok_cols, r2_threshold


In [438]:
def many_shuffles(num=10):
  bestestbest = [['SalePrice', 'OverallQual'], 0.67]
  for x in range(num):
    print('shuffle:', x+1)
    result = get_best_cols(df_cats, ['SalePrice', 'OverallQual'])
    if result[1] > bestestbest[1]:
      bestestbest = result
      print(bestestbest)
  print('\n\nThe result is:')
  print(bestestbest)

In [439]:
def drop_n_log(df):
  df_num = df.copy()
  #Create a new parameter that combines all porches
  df_num['PorchSF'] = df_num['OpenPorchSF'] + df_num['EnclosedPorch'] + df_num['3SsnPorch'] + df_num['ScreenPorch']

  #drop low numbers 
  #cols = ['KitchenAbvGr', 'BedroomAbvGr']
  #df_num = drop_low_zeros(df_num, cols)

  df_num['GarageYrBlt'] = df_num['GarageYrBlt'].fillna(df_num['GarageYrBlt'].mean()) #REPLACE

  #Drop numeric columns
  df_num = df_num.drop(columns=['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']) #drop porches
  df_num = df_num.drop(columns=['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF'])                    #drop basements
  df_num = df_num.drop(columns=['GarageCars'])                                               #drop garage
  df_num = df_num.drop(columns=['LowQualFinSF', 'PoolArea', 'MiscVal', 'TotRmsAbvGrd', '1stFlrSF'])      #drop misc

  #wierdly it produced worse result
  for column in [col for col in df_num.columns if df_num[col].dtype not in ['category', 'object']]:
    df_num[column] = df_num[column]+1
  #  df_num[column] = np.log(df_num[column] + 1)

  to_log = ['LotFrontage', 'LotArea', 'GrLivArea', 'SalePrice']#, 'GarageArea']#,'WoodDeckSF', 'MasVnrArea','TotalBsmtSF''2ndFlrSF', 'PorchSF']#]#,]#, ]#, ]#]]
  for col in to_log:
    df_num[col] = np.log(df_num[col])
  print(df_num.shape)
  return df_num

In [440]:
#Cut iqr
def make_iqr(df_num):
  df_iqr = df_num.copy()
  for col in ['SalePrice', 'LotFrontage', 'LotArea', 'MasVnrArea', \
              'TotalBsmtSF', 'GrLivArea', \
              'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageArea', 'PorchSF']:

      df_iqr = cut_iqr(df_iqr, col, 3)
      #plotme(df_iqr, col)

  print(df_iqr.shape)
  return df_iqr

In [441]:
def make_cats(df_iqr):
  df_cats = df_iqr.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'Utilities', 'Condition1', \
                        'Condition2', 'Street', 'LandContour', 'LandSlope', \
                  'RoofMatl', 'ExterCond', 'BsmtCond', 'BsmtFinType2', 'Heating', 'CentralAir', \
                  'Electrical', 'Functional', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType'], axis = 1)
  df_cats = create_mt_catcol(df_cats)
  print(df_cats.shape)
  return df_cats

In [442]:
df_num = drop_n_log(df)

(1460, 68)


In [443]:
df_iqr= make_iqr(df_num)

(1140, 68)


In [444]:
df_cats = make_cats(df_iqr)

(1140, 46)


In [445]:
#Find second best parameter
valids = get_valid_cols(df_cats, ['SalePrice', 'OverallQual'])
find_best_param(df_cats, valids, ['SalePrice','OverallQual'])

['GrLivArea', 0.783920660454508]

In [446]:
split_run_test(df_cats[['SalePrice', 'OverallQual', 'GrLivArea', 'TotalBsmtSF', 'LotArea', 'YearBuilt', 'OverallCond', 'GarageArea', 'BsmtFullBath', 'Fireplaces', 'BedroomAbvGr', 'YearRemodAdd', 'HalfBath']])

train: r2 0.886399882244371
train: root mean squared error: 0.133150478672193
Test:
test: r2 0.9171824387890771
test: root mean squared error: 0.11180536360069847



In [447]:
#get best combination 
get_best_cols(df_cats, ['SalePrice', 'OverallQual'])

(['SalePrice',
  'OverallQual',
  'GrLivArea',
  'TotalBsmtSF',
  'GarageArea',
  'YearBuilt',
  'OverallCond',
  'Fireplaces',
  'LotFrontage',
  'BedroomAbvGr',
  'HalfBath',
  'YrSold',
  'WoodDeckSF',
  'MasVnrArea',
  'BsmtHalfBath',
  'YearBuilt'],
 0.9035600290251458)

In [449]:
#get best orders of best combinations
many_shuffles(5)

shuffle: 1
(['SalePrice', 'OverallQual', 'GrLivArea', 'TotalBsmtSF', 'GarageArea', 'LotArea', 'YearBuilt', 'OverallCond', 'Fireplaces', 'BsmtFullBath', 'BedroomAbvGr', 'HalfBath', 'YrSold', 'MasVnrArea', 'BedroomAbvGr'], 0.9164415567934873)
shuffle: 2
shuffle: 3
(['SalePrice', 'OverallQual', 'GrLivArea', 'TotalBsmtSF', 'GarageArea', 'LotArea', 'YearBuilt', 'OverallCond', 'Fireplaces', 'BsmtFullBath', 'BedroomAbvGr', 'HalfBath', 'YearBuilt', 'YearRemodAdd'], 0.9180531748470215)
shuffle: 4
shuffle: 5
shuffle: 6
shuffle: 7
shuffle: 8
shuffle: 9
shuffle: 10


The result is:
(['SalePrice', 'OverallQual', 'GrLivArea', 'TotalBsmtSF', 'GarageArea', 'LotArea', 'YearBuilt', 'OverallCond', 'Fireplaces', 'BsmtFullBath', 'BedroomAbvGr', 'HalfBath', 'YearBuilt', 'YearRemodAdd'], 0.9180531748470215)


train: r2 0.886399882244371

train: root mean squared error: 0.133150478672193

test: r2 0.9171824387890771

test: root mean squared error: 0.11180536360069847


'SalePrice', 'OverallQual', 'GrLivArea', 'TotalBsmtSF', 'LotArea', 'YearBuilt', 'OverallCond', 'GarageArea', 'BsmtFullBath', 'Fireplaces', 'BedroomAbvGr', 'YearRemodAdd', 'HalfBath'

log of 'LotFrontage', 'LotArea', 'GrLivArea', 'SalePrice'

['SalePrice', 'OverallQual', 'GrLivArea', 'TotalBsmtSF', 'LotArea', 'YearBuilt', 'OverallCond', 'GarageArea', 'BsmtFullBath', 'Fireplaces', 'BedroomAbvGr', 'YearRemodAdd', 'HalfBath']

log of ['LotFrontage', 'LotArea', 'GrLivArea', 'SalePrice','GarageArea']